In [1]:
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

from langchain_experimental.agents.agent_toolkits import create_csv_agent

In [3]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [4]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [5]:
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q5_K_M.gguf",
    temperature=0.00001,
    max_tokens=2000,
    n_ctx=10000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [6]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [17]:
question = "Generate a recipe with  the following ingredients:- Potato, Chicken,onion , tomato , cashew, butter, thyme , honey and artichoke "
llm_chain.run(question)


Step 1: Identify the main dish you want to make with these ingredients. Based on the ingredients provided, it seems like you want to make a hearty and flavorful chicken dish. How about a Chicken and Artichoke Casserole?
Step 2: Gather all the necessary ingredients. Here are the ingredients we have so far:
* Potato
* Chicken
* Onion
* Tomato
* Cashew
* Butter
* Thyme
* Honey


KeyboardInterrupt: 

In [ ]:
llm("Give me a poem on Ferrari")

In [ ]:
llm("Generate a recipe with  the following ingredients:- Potato, Chicken,onion ")

In [7]:
from langchain.document_loaders.csv_loader import CSVLoader

In [8]:
loader = CSVLoader(file_path="total1.csv")

In [9]:
data = loader.load()

In [10]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [11]:
from langchain.vectorstores import Chroma 

vector_store = Chroma.from_documents(data , embedding = embed_model)

In [12]:


from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='refine',
    retriever=vector_store.as_retriever())

In [13]:
rag_pipeline("Generate a recipe considering I have the following ingredients :- Chicken, Onion , carrot , celery , honey , ham and flour")



Llama.generate: prefix-match hit



Please let me know if you need any further clarification or details.

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


{'query': 'Generate a recipe considering I have the following ingredients :- Chicken, Onion , carrot , celery , honey , ham and flour',
 'result': ''}

In [14]:
question = "Generate a recipe considering I have the following ingredients :- Chicken, Onion , carrot , celery , honey , ham and flour"
docs = vector_store.similarity_search(question)
#result = llm_chain(docs)
docs

[Document(page_content=": 21683\nRecipe Title: 30-Minute Spicy Pork and Sweet Potato Stew\nIngredients: ['1 cup canned unsalted diced tomatoes', '1 medium white onion, quartered', '2 cloves garlic', '1/2 chipotle in adobo sauce, plus 1 tablespoon of the sauce', '2 tablespoons vegetable oil', '1 large pork tenderloin, trimmed and cut into 1-inch chunks (about 1 1/2 pounds)', 'Kosher salt and freshly ground black pepper', '1 tablespoon chili powder', 'One 8-ounce sweet potato, peeled and cut into 1/2-inch chunks', '2 cups low-sodium chicken broth', '1 tablespoon pickled jalapenos', '1 cup 2 percent Greek yogurt', '1/4 medium head red cabbage', '1 medium Haas avocado', '1 lime', '1/2 cup loosely packed fresh cilantro', 'Eight 6-inch corn tortillas, warmed']\nSteps: {'Step 1': 'Puree the tomatoes, onions, garlic, chipotle and adobo sauce in a food processor until completely smooth.', 'Step 2': 'Heat the oil in a large Dutch oven or pot over medium-high heat until nearly smoking. Add the po

In [ ]:
len(docs[0])

TypeError: object of type 'Document' has no len()